In [1]:
import os

In [2]:
%pwd

'd:\\college_work\\4th year\\Sem7th\\Project\\OBE\\Paper- Blooms taxanomy\\Boolm-s-Level-Detection-A-MLOPS-Project\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\college_work\\4th year\\Sem7th\\Project\\OBE\\Paper- Blooms taxanomy\\Boolm-s-Level-Detection-A-MLOPS-Project'

In [4]:
import pandas as pd
df = pd.read_csv("./artifacts/data_ingestion/blooms_dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    600 non-null    object
 1   Level   600 non-null    object
dtypes: object(2)
memory usage: 9.5+ KB


In [5]:
from pathlib import Path 
from dataclasses import dataclass

In [6]:
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_path: Path
    STATUS_FILE: str 
    all_schema: dict

In [7]:
from blooms.utils.common import read_yaml,create_directories
from blooms.constants import *


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            STATUS_FILE=config.STATUS_FILE,
            root_dir=config.root_dir,
            unzip_data_path=config.unzip_data_path,
            all_schema=schema,
        )
        return data_validation_config

In [9]:
from blooms import logger 
import os 
import pandas as pd

In [10]:
class DataValidation:
    def __init__(self,config:DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try: 
            validation_status = None 
            data = pd.read_csv(self.config.unzip_data_path)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False 
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation Status: {validation_status}")
                else:
                    validation_status = True 
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validattion Status: {validation_status}")

                return validation_status
        except Exception as e:
            raise e

In [11]:
try: 
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-12-14 19:55:37: INFO: common: yaml file: config\config.yaml loaded succcessfully]
[2023-12-14 19:55:37: INFO: common: yaml file: params.yaml loaded succcessfully]
[2023-12-14 19:55:37: INFO: common: yaml file: schema.yaml loaded succcessfully]
[2023-12-14 19:55:37: INFO: common: created directory at : artifacts]
[2023-12-14 19:55:37: INFO: common: created directory at : artifacts/data_validation]
